In [1]:
import copy
import itertools

import numpy as np
import plotly.offline as pty
import plotly.graph_objs as ptygo
import plotly.grid_objs as ptygr
import plotly.tools as ptytools
import scipy.stats as st

import FilterPointCloud
import ProcessPlanePointCloud
import PLYObject

pty.init_notebook_mode(connected=True)

In [2]:
pcDataNear01 = ProcessPlanePointCloud.main('RealSense-D415-Data-01-Near/Laser/', {
    2.5: [
        'x>-1.1279',
        'x<1.1447',
        'y<0.58',
    ]
})

Processing folder RealSense-D415-Plane-0.5m-1280x720-6-Laser
Filters: []
n = 4
e = [0.001272743630753398, 0.0011975694776483116, 0.0012006031493034745, 0.0011872592007701884]
d = [3599376.6029766873, 3673032.553299536, 3674125.8386335033, 3689355.2219764963]

Processing folder RealSense-D415-Plane-0.5m-640x480-6-Laser
Filters: []
n = 5
e = [0.000979629493735353, 0.0009964415014598834, 0.000989604462056468, 0.0009927091579821573, 0.0009845015557435753]
d = [1620227.2688009036, 1625838.3362104876, 1625583.8186036807, 1623391.854621434, 1626158.704114806]

Processing folder RealSense-D415-Plane-1.0m-1280x720-6-Laser
Filters: []
n = 5
e = [0.0041715439219746735, 0.003960595703093468, 0.0039253768398255905, 0.0038850448900966386, 0.0038860758240216128]
d = [871253.1868182201, 869398.9876419047, 866467.4511905949, 866021.162053019, 870413.9585067385]

Processing folder RealSense-D415-Plane-1.0m-640x480-6-Laser
Filters: []
n = 5
e = [0.00229134996822316, 0.002204437999996277, 0.00224646557998

In [3]:
pcDataNear02 = ProcessPlanePointCloud.main('RealSense-D415-Data-02-Near/Laser', {
    2.0: [
        'x>-0.925',
        'x<1.022',
    ],
    2.5: [
        'x>-0.695',
        'x<1.26',
        'y>-0.88',
    ],
})

Processing folder RealSense-D415-Plane-0.5m-1280x720-6-Laser
Filters: []
n = 10
e = [0.0013949282841351812, 0.0014255677970811049, 0.0014325409772281482, 0.0014249097007819063, 0.0014376540565011512, 0.0014199174841931853, 0.0014235287548650182, 0.0014179723438365382, 0.0014206599380354418, 0.0014177412964936106]
d = [3356355.665512987, 3405366.2572091273, 3382880.307472585, 3396178.836325475, 3390912.999698296, 3393509.2750907377, 3405477.7994173313, 3386404.7944177, 3390380.077580024, 3395075.552177173]

Processing folder RealSense-D415-Plane-0.5m-424x240-6-Laser
Filters: []
n = 10
e = [0.0011942200810833598, 0.0011666447463927851, 0.0011564657041198386, 0.0011361390377385098, 0.0011375189999655976, 0.0011248271268834826, 0.0011350265720092081, 0.0011445617828204785, 0.001132418403249444, 0.001120339177279971]
d = [382867.72538608825, 383337.8711013015, 383505.1233343652, 383339.09719987033, 382607.2879608115, 383328.1837409342, 383189.15687336103, 383657.0026579736, 383155.585349071

In [4]:
pcDataFar02 = ProcessPlanePointCloud.main('RealSense-D415-Data-02-Far/Laser', {
    5.0: [
        'x>-0.43816',
        'x<0.44257',
        'y>-0.93533',
        'y<1.0309',
    ],
    7.5: [
        'x>-0.39332',
        'x<0.28466',
        'y>-0.7516',
        'y<1.0035',
    ],
    10.0: [
        'x>-0.58987',
        'x<0.3114',
        'y>-0.5276',
        'y<1.1418',
    ],
})

Processing folder RealSense-D415-Plane-10.0m-1280x720-6-Laser
Filters: [X>-0.5899, X<0.3114, Y>-0.5276, Y<1.1418]
n = 10
e = [0.23085666893709922, 0.2077330722230642, 0.2199522560400499, 0.21882488932987829, 0.22423502473713103, 0.21776178656060136, 0.21766521722725402, 0.18927177475116874, 0.2205101981142895, 0.2087043304777838]
d = [2574.9720668119367, 1302.3552482870157, 1542.5684834687559, 2222.333823700798, 1447.6119245279717, 1806.0421798080026, 2161.43952370695, 1370.3442954585869, 1855.0183075703314, 1495.8761048218416]

Processing folder RealSense-D415-Plane-10.0m-640x480-6-Laser
Filters: [X>-0.5899, X<0.3114, Y>-0.5276, Y<1.1418]
n = 10
e = [0.2184192989724593, 0.2146084483158834, 0.22504543936565913, 0.2088490778913895, 0.22728890772147797, 0.22373159247697524, 0.20909972081134232, 0.19647075912550815, 0.22380815616275923, 0.22635438706810146]
d = [1487.570827977771, 1071.188451569655, 890.181249230208, 938.6665539563201, 895.9296799345043, 1026.0848181330562, 986.8528935336

In [5]:
def ptyColorMap(names):
    ptyColors = itertools.cycle(reversed(ptytools.DEFAULT_PLOTLY_COLORS))

    return dict(zip(names, ptyColors))

In [6]:
def createGraphObjectsFromPlaneData(data):
    goData = {}
    layoutData = {}
    
    colorMap = ptyColorMap(data.keys())
    
    measurandMap = {
        #'planeFitError': {
        #    'xaxis': {
        #        'title': 'Distance [m]',
        #    },
        #    'yaxis': {
        #        'title': 'Error [mm]',
        #    },
        #    'title': 'Plane Fit Error',
        #},
        'planeFitErrorStd': {
            'xaxis': {
                'title': 'Distance [m]',
            },
            'yaxis': {
                'title': 'Error STD [mm]',
                'range': [0, 7.3],
            },
            'title': 'Plane Fit Error Standard Deviation',
            'factor': 100,
        },
        'pointDensity': {
            'xaxis': {
                'title': 'Distance [m]',
            },
            'yaxis': {
                'title': 'Point Density [p / m²]',
                'tickformat': ',d',
                'hoverformat': ',.2f',
            },
            'title': 'Point Density',
            'factor': 1,
        },
    }
    
    resolutions = [*map(lambda res: 'x'.join([*map(str, res)]), sorted(map(lambda res: [*map(int, res.split('x'))], data.keys()), reverse=True))]
    
    for res in resolutions:
        showlegend = True
        
        for measurand in data[res]:
            if measurand not in measurandMap:
                continue
            
            distances = sorted(data[res][measurand].keys())
            errors = [np.array(data[res][measurand][dist]) * measurandMap[measurand]['factor'] for dist in distances]
            
            go = ptygo.Scatter(
                x=distances,
                y=[errorMeasurand.mean() for errorMeasurand in errors],
                error_y=dict(
                    type='data',
                    visible=True,
                    symmetric=True,
                    array=np.abs(np.array([st.t.interval(0.95, errorMeasurand.shape[0]-1, loc=0, scale=st.sem(errorMeasurand)) for errorMeasurand in errors])[:, 0]),
                    color=colorMap[res],
                ),
                name=res,
                legendgroup=res,
                showlegend=showlegend,
                line=dict(
                    color=colorMap[res],
                ),
            )
            
            if measurand not in goData:
                goData[measurand] = []
            
            goData[measurand].append(go)
            showlegend = False
            
            layoutData[measurand] = copy.deepcopy(measurandMap[measurand])
            
            if 'range' in layoutData[measurand]['yaxis']:
                layoutData[measurand]['yaxis']['range'][0] = min(0, *[data.min() for data in errors])
                layoutData[measurand]['yaxis']['range'][1] = max(7.3, *[data.max() for data in errors])
    
    return goData, layoutData

In [7]:
def plotPlaneData(data, title):
    go, lay = createGraphObjectsFromPlaneData(data)
    goKeys = [*go.keys()]
    rows = int(np.floor(np.sqrt(len(go))))
    cols = int(np.ceil(np.sqrt(len(go))))
    
    titles = tuple(lay[goKey]['title'] for goKey in goKeys)
    
    fig = ptytools.make_subplots(rows=rows, cols=cols, subplot_titles=titles, print_grid=False)
    fig['layout']['title'] = title
    fig['layout']['separators'] = ',.'
    
    for v in range(rows):
        for u in range(cols):
            i = v * rows + u
            
            if i >= len(go):
                break
            
            for t in go[goKeys[i]]:
                fig.append_trace(t, v + 1, u + 1)
            
            layout = lay[goKeys[i]]
            
            if 'xaxis' in layout:
                fig['layout']['xaxis{:d}'.format((v+1)*(rows-1)+u+1)].update(layout['xaxis'])
            
            if 'yaxis' in layout:
                fig['layout']['yaxis{:d}'.format((v+1)*(rows-1)+u+1)].update(layout['yaxis'])
    
    
    pty.iplot(fig)

In [8]:
plotPlaneData(pcDataNear01, 'Plan Fitting Near - Dataset 01')

In [9]:
plotPlaneData(pcDataNear02, 'Plan Fitting Near - Dataset 02')

Large divergence for different resolutions results in different aspect ratio. Apparently, the device just defines a region of interest for out of the full resolution depth map. This results in lesser artifacts in the corners.

In [10]:
plotPlaneData(pcDataFar02, 'Plan Fitting Far - Dataset 01')

The error values and plots tend to show an improvement for increasing distance again. This is against intuitive belief. Further digging in the data reveal that the error in this distances is so large, the plane is fittet completely wrong. One could account for that by increasing the area of the target plane in the image. Due to limitations in our experiment setup, we couldn't find a large enough plane which still fitted indoors to cover the whole field of view at a distance of 10 meters.

In [ ]:
filt = FilterPointCloud.parseFilters([
    'x>-0.7188',
    'x<1.26',
    'y>-0.88',
])
ply = PLYObject.PLYObject(r'RealSense-D415-Data-02-Near\Laser\RealSense-D415-Plane-2.5m-424x240-6-Laser\1.pointcloud.ply')
fply = FilterPointCloud.filterPLYObject(ply, filt)

vert = ply.getVertices()
fvert = fply.getVertices()

data = [
    ptygo.Scatter3d(
        x=fvert[0],
        y=fvert[1],
        z=fvert[2],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.7,
        ),
    )
]
pty.iplot(data)

In [ ]:
data = [
    ptygo.Scatter3d(
        x=vert[0],
        y=vert[1],
        z=vert[2],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.7,
        ),
    )
]
pty.iplot(data)